## Сбор данных

*Подключение библиотек для сбора данных*

In [ ]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By

*Создание списка с городами, по которым наш парсер будет проходиться*

In [ ]:
CITY = [
    "moskva", "sankt-peterburg", "vladimir", "volgograd", "voronezh", "ekaterinburg",
    "ivanovo", "kazan", "kaluga", "kostroma", "krasnodar", "krasnoyarsk",
    "nizhniy_novgorod", "novosibirsk", "omsk", "perm", "rostov-na-donu", "samara",
    "saratov", "tver", "tula", "ufa", "chelyabinsk", "yaroslavl"
]

*Создание словаря, где ключ - это признак, с помощью которого мы будем искать данные на странице, а значение - название столбца, куда будет записываться значение, соответствующее признаку*

In [ ]:
dict_characteristics_main = {
    "Год выпуска": "year of production",
    "Пробег": "mileage",
    "Цвет": "exterior colour",
    "Налог": "tax",
    "Коробка": "transmission",
    "Привод": "wheel drive",
    "Состояние": "condition",
    "Владельцы": "number of owners"
}

dict_characteristics_other = {
    "Объем": "engine volume",
    "Мощность": "engine power",
    "Тип двигателя": "engine type",
    "Топливо": "fuel type",
    "Разгон": "acceleration",
    "Расход": "MPG",
    "Страна марки": "country of the car brand",
    "Класс автомобиля": "class",
    "Максимальная скорость": "top speed",
    "Экологический класс": "environmental class"
}

*Генерируем URL для страницы с объявлениями об автомобилях по указанному городу и номеру страницы*

In [ ]:
def get_url(city, page):
    return f"https://auto.ru/{city}/cars/used/?page={page}"

*Получаем количество страниц с объявлениями об автомобилях в конкретном городе*

In [ ]:
def get_pagenation(driver):
    return int(driver.find_element(by = By.XPATH, value = "//a[contains(@class, 'ListingPagination__page')][last()]").get_attribute("href").split("page=")[1])

*Извлекаем URL-адреса объявлений об автомобилях с текущей страницы и записываем их в список*

In [ ]:
def get_urls_from_page(driver):
    array_auto = driver.find_elements(by = By.XPATH, value = "//a[@class='Link ListingItemTitle__link']")
    array_url_auto = []
    for i in range(len(array_auto)):
        array_url_auto.append(array_auto[i].get_attribute("href"))
    return array_url_auto

*Проходимся поочерёдно по каждому городу, странице и сохраняем ссылки на объявления, вызываем вспомогательную функцию, которая найдёт все характеристики на странице*

In [ ]:
def processing_autos(driver):
    for city in CITY:
        driver.get(get_url(city, 1))
        pages = get_pagenation(driver)
        for page in range(1, pages + 1):
            array_informations_auto = []
            driver.get(get_url(city, page))
            array_url_car = get_urls_from_page(driver)
            for i in range(len(array_url_car)):
                array_informations_auto.append({})
                try:
                    driver.get(array_url_car[i])
                except TimeoutException:
                    pass
                driver.implicitly_wait(1)
                try:
                    processing_characteristics_auto(driver, array_informations_auto, i)
                except (NoSuchElementException, TimeoutException):
                    pass

            array_informations_auto.clear()

*Вспомогательная функция*

In [ ]:
def processing_characteristics_auto(driver, array_informations_auto, i):
    model = driver.find_element(by = By.XPATH, value = "//h1[@class='CardHead__title']")
    array_informations_auto[i]["model auto"] = model.text

    price = driver.find_element(by = By.XPATH, value = "//span[@class='OfferPriceCaption__price']")
    array_informations_auto[i]["price"] = price.text.replace('₽', '')

    generation = driver.find_element(by = By.XPATH, value = "//span[contains(text(), 'Поколение')]/../span[@class='CardInfoRow__cell'][2]//a")
    url = generation.get_attribute("href")

    for key, value in dict_characteristics_main.items():
        pair = driver.find_elements(by = By.XPATH, value = f"//span[contains(text(), '{key}')]/../span[@class='CardInfoRow__cell']/../span[2]")
        try:
            if (key == "Налог"):
                array_informations_auto[i][f"{value}"] = pair[0].text.replace('₽', '')
            else:
                array_informations_auto[i][f"{value}"] = pair[0].text
        except IndexError:
            array_informations_auto[i][f"{value}"] = "?"

    driver.get(f'{url}')

    for key, value in dict_characteristics_other.items():
        pair = driver.find_elements(by=By.XPATH, value=f"//span[contains(text(), '{key}')]/..//span[2]")
        try:
            array_informations_auto[i][f"{value}"] = pair[0].text
        except IndexError:
            array_informations_auto[i][f"{value}"] = "?"

*Отключаем обнаружение WebDriver, создаём экземпляр веб-драйвера Firefox и запускаем наш парсер*

In [ ]:
options = webdriver.FirefoxOptions()
options.set_preference("dom.webdriver.enabled", False)

driver = webdriver.Firefox(options=options)

driver.close()